# Playground for parsing and extracting information from PDFs - Llama Index


## Parsing PDF document using LlamaParse

LlamaParse is a highly accurate parser for complex documents like financial reports, research papers, and scanned PDFs. It handles images, tables, and charts wiht ease.

It transforms complex documents into text, markdown, or JSON formats.

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
import os

faq_file = os.path.abspath("../pdfs/novo_regime.pdf")

parser = LlamaParse(result_type="text")

file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(
    input_files=[faq_file],
    file_extractor=file_extractor
    ).load_data()

documents

/Users/patriciaoliveira/repos/ai-assistant-self-employers-pt/.venv/lib/python3.12/site-packages/llama_index/download/module.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Started parsing the file under job_id 2e7332ff-ed63-4a70-abae-ee21f405427e


[Document(id_='7433d071-72f7-4c70-8467-2b332d44a3c1', embedding=None, metadata={'file_path': '/Users/patriciaoliveira/repos/ai-assistant-self-employers-pt/pdfs/novo_regime.pdf', 'file_name': 'novo_regime.pdf', 'file_type': 'application/pdf', 'file_size': 486698, 'creation_date': '2025-07-28', 'last_modified_date': '2025-07-28'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='O limite mínimo da base de incidência contributiva dos trabalhadores independentes abrangidos pelo\nregime de contabilidade organizada é 1,5 vezes o valor do IAS.\nnovos\n\n\nPerguntas Frequentes\n\n\nNOVO REGIME DOS TRABALHADORES INDEPENDENTES\n\nINSTITUTO DA SEGURAN

In [3]:
from llama_index.core import VectorStoreIndex

# Create an index from the documents. The VectorStoreIndex converts the parsed documents into a vectorized format that can be queried.
# This index will allow us to perform semantic searches over the content of the documents.
index = VectorStoreIndex.from_documents(documents)

# It creates a query search engine from the index.
# This engine will allow us to query the index using natural language questions and returns the most relevant results.
query_engine = index.as_query_engine()

query = "Como é determinado o rendimento?"
response = query_engine.query(query)
print(response)


O rendimento é determinado com base nos rendimentos obtidos nos três meses imediatamente anteriores ao mês da declaração trimestral, de acordo com percentagens específicas para diferentes tipos de rendimentos. Para os trabalhadores independentes abrangidos pelo regime de contabilidade organizada, o rendimento relevante corresponde ao lucro tributável apurado no ano civil imediatamente anterior, declarado no Anexo SS da Declaração Modelo 3 do IRS.


## Extracting information from PDF document using Structured Output

Llama Index framework provides two ways to extract information from PDFs:
1. Llama Cloud - [LlamaExtract](https://docs.cloud.llamaindex.ai/llamaextract/getting_started/python)
2. Structured LLMs - [Structured Data Extraction](https://docs.llamaindex.ai/en/stable/understanding/extraction/structured_llms/)

### Common Dependencies

Pydantic class that represents FAQs structured output.

In [10]:
from pydantic import BaseModel, Field

class FAQs(BaseModel):
    question: str = Field(description="The question to be answered.")
    answer: str = Field(description="The answer to the question.")

### LlamaExtract

This tool transforms complex documents into well-types structured data with:
- Customizable extraction agents and schemas
- Batch processing capabilities for scale
- Iterative schema development

**LlmaExtract** provides a simple API for extracting structured data from unstructured documents like PDFs, text files, and images.

In [11]:
from llama_cloud_services import LlamaExtract
import os

In [12]:
prompt = """
You are an AI that extracts structured data from text. The FAQ contains a list of questions and answers. There are a total of 55 questions, so make sure you extract all question and answers.

Extraction Rules:
1. Every question starts with an incremental number followed by a dot and ends with a question mark.

2. The answer to that question starts with R, (letter "R" followed by a comma) and continues until the next question or the end of the document.

3. Return a JSON array where each element has:
- question: the full question text.
- answer: the full answer text.

Extract ONLY the questions and answers following these rules. Ignore any other text.
"""

extractor = LlamaExtract()
agents = [agent.name for agent in extractor.list_agents()]

if not "FAQs Seguranca Social" in agents:
    agent = extractor.create_agent(
        name="FAQs Seguranca Social",
        data_schema=FAQs,
        config={
            "extraction_target": "PER_PAGE",
            "extraction_mode": "MULTIMODAL",
            "multimodal_fast_mode": false,
            "system_prompt": prompt,
            "use_reasoning": false,
            "cite_sources": false,
            "confidence_scores": false,
            "chunk_mode": "PAGE",
            "high_resolution_mode": false,
            "invalidate_cache": false,
            "page_range": "3-27"
        }  
    )

agent = extractor.get_agent("FAQs Seguranca Social")
pdf_path = os.path.abspath("../pdfs/novo_regime.pdf")
result = agent.extract(pdf_path)
print(result.data)

Extracting files: 100%|██████████| 1/1 [00:21<00:00, 21.40s/it]

[{'question': '1. Quem está abrangido pelo novo regime dos trabalhadores independentes?', 'answer': 'R, Estão abrangidos pelo novo regime dos trabalhadores independentes todos os trabalhadores que exerçam atividade por conta própria, incluindo empresários em nome individual, titulares de estabelecimentos individuais de responsabilidade limitada, profissionais liberais, entre outros, que não estejam expressamente excluídos por lei.'}, {'question': '1.  Quando produz efeito o enquadramento no regime?', 'answer': 'R: O primeiro enquadramento no regime dos trabalhadores independentes só produz efeitos no 1.° dia do 12.º mês posterior ao do início de atividade.\n\nExemplo:\n\nSe o trabalhador independente iniciou a sua atividade a 1 de março de 2024, o seu enquadramento produz efeitos a 1 de março de 2025.\n\nSe o trabalhador independente iniciar a sua atividade independente na Autoridade Tributária e Aduaneira a 10 de janeiro de 2024, o seu enquadramento produz efeitos a 1 de janeiro de 20

In [13]:
for item in result.data:
    print(f"Question: {item['question']}")
    print(f"Answer: {item['answer']}")
    print("-" * 40)

Question: 1. Quem está abrangido pelo novo regime dos trabalhadores independentes?
Answer: R, Estão abrangidos pelo novo regime dos trabalhadores independentes todos os trabalhadores que exerçam atividade por conta própria, incluindo empresários em nome individual, titulares de estabelecimentos individuais de responsabilidade limitada, profissionais liberais, entre outros, que não estejam expressamente excluídos por lei.
----------------------------------------
Question: 1.  Quando produz efeito o enquadramento no regime?
Answer: R: O primeiro enquadramento no regime dos trabalhadores independentes só produz efeitos no 1.° dia do 12.º mês posterior ao do início de atividade.

Exemplo:

Se o trabalhador independente iniciou a sua atividade a 1 de março de 2024, o seu enquadramento produz efeitos a 1 de março de 2025.

Se o trabalhador independente iniciar a sua atividade independente na Autoridade Tributária e Aduaneira a 10 de janeiro de 2024, o seu enquadramento produz efeitos a 1 de 

### Structured Data Extraction

The highest-level way to extract structured data in LlmaIndex is to instantiate a Structured LLM. It refers to the practice of guiding LLMs to produce output in a defined format, like JSON or SML, rather than free-form test.

This structured output is more easily parsed and utilized by other systems, making LLM-driven pipelines more deterministic and efficient.

In [14]:
from llama_index.readers.file.docs.base import PDFReader
from pathlib import Path

pdf_reader = PDFReader()
documents = pdf_reader.load_data(file=pdf_path)

documents[0].text.strip()

'O limite mínimo da base de incidência contributiva dos trabalhadores independentes abrangidos pelo \nregime de contabilidade organizada é 1,5 vezes o valor do IAS.   \nnovos   \nPerguntas Frequentes \n  \n \nNOVO REGIME DOS TRABALHADORES INDEPENDENTES \n \n \n \nINSTITUTO DA SEGURANÇA SOCIAL, I.P'

In [15]:
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts import PromptTemplate

llm = OpenAI(model="gpt-4o")
prompt_template = PromptTemplate("You are an AI that extracts structured data from text. The FAQ contains a list of questions and answers: '{document}'. There are a total of 55 questions, so make sure you extract all question and answers. Extraction Rules: Every question starts with an incremental number followed by a dot and ends with a question mark. The answer to that question starts with R, (letter "R" followed by a comma) and continues until the next question or the end of the document.")

response = llm.structured_predict(
    FAQs,
    prompt_template,
    document=documents[0].text.strip()
)